In [1]:
import os
import django
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time

In [2]:
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'web_restaurant.settings')
django.setup()

In [5]:
from restaurant.models import Restaurant, Review, Chef

# Chrome 드라이버 설정
options = Options()
options.add_argument("--headless")  # 브라우저 창을 띄우지 않고 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# 비동기 코드에서 ORM 호출 허용
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https://app.catchtable.co.kr/ct/shop/neo_/review?type=DINING&sortingFilter=L')


# 페이지가 로딩될 시간을 기다림
time.sleep(3)

# 페이지 스크롤 함수
def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 스크롤 후 대기 시간

        # 새로운 높이 계산 후 비교
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 스크롤이 더 이상 불가능할 때 종료
        last_height = new_height

# 스크롤을 끝까지 내림
scroll_to_bottom()

# <br> 태그 제거 함수
def clean_review_text(text):
    return re.sub(r'<br\s*/?>', '\n', text)  # <br>을 줄바꿈으로 변환

# 평점과 리뷰 내용 수집
reviews = []
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "_10fm75h6"))
    )
    # 모든 평점과 리뷰 요소를 찾음
    ratings = driver.find_elements(By.CLASS_NAME, "_10fm75h6")
    contents = driver.find_elements(By.CLASS_NAME, "review-content")

    for rating, content in zip(ratings, contents):
        # rating 값 정수로 버림
        rating_value = float(rating.text)
        review_type = 'good' if rating_value == 5 else 'bad'
        review_text = clean_review_text(content.get_attribute("innerHTML").strip())

        review_data = {
            "rating": review_type,
            "content": review_text
        }
        reviews.append(review_data)

except Exception as e:
    print(f"오류 발생: {e}")




restaurant_name = '식당네오'

# test용 셰프, 식당 생성 코드 (추후 삭제)
chef, created = Chef.objects.get_or_create(chef_name = '최강록', defaults={'image_url': "./최강록.jpeg"})
restaurant = Restaurant.objects.create(restaurant_name=restaurant_name, chef=chef)


# 원래는 이 부분 활성화
# restaurant = Restaurant.objects.get(restaurant_name=restaurant_name)


# 리뷰 저장
for review in reviews:
    Review.objects.create(
        restaurant=restaurant,
        review_text=review['content'],
        review_category=review['rating']
    )
    # 평점, 리뷰 출력
    print(f"평점: {review['rating']}")
    print(f"리뷰: {review['content']}")

print("리뷰 저장 완료")

# 드라이버 종료
driver.quit()



KeyboardInterrupt: 

In [24]:
from restaurant.models import Review
print(Review)

<class 'restaurant.models.Review'>
